In [4]:
import csv
import numpy as np 
import matplotlib.mlab as mlab 
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from Bio.Seq import Seq
from Bio import SeqIO
import re
import pandas as pd
import collections
from collections import defaultdict
import os
import statistics
from matplotlib import rc
import matplotlib.pyplot as plt

In [ ]:
def check_size_align(a,b) :
    size=0
    if a > b :
        size =a-b
    else :
        size =b-a
    return size
def check_presence(elt,dic):
    if elt in dic :
        return 1
    else :
        return 0
def check_duplication(info_file,threshold,dic_occurrence): 
    inpt=csv.reader(open(info_file,"r"),delimiter="\t")
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    count=0
    seen_validate=set()
    dic=defaultdict(list)
    dic_last=defaultdict(float)
    black_list=[]
    dic_segm=defaultdict(float)
    dic_td=defaultdict(float)
    dic_not_valid=defaultdict(float)
    count_td=0
    liste_segm_dup=[]
    count_segm=0
    dic_tes=defaultdict(list)
    for elt in inpt :
        if len(elt[9].split("_"))<=4 :
            match=int(elt[0])
            size_align=check_size_align(int(elt[12]),int(elt[11]))
            size_align_ref=check_size_align(int(elt[16]),int(elt[15]))
            size_inser=int(elt[9].split("_")[-1])
            chrom_inser=re.sub("chr", "",elt[9].split("_")[0])
            inser_pos=int(elt[9].split("_")[2])
            chrom_ref=re.sub("chr", "",elt[13])
            ref_pos=int(elt[15])
            head=chrom_inser+"_"+str(inser_pos)+"_"+str(size_inser)
            #print(head)
            #print(size_align,size_istsnser,chrom_inser,inser_pos,chrom_ref,ref_pos)
            #percentage=float(size_align/size_inser)
            percentage=float(match/size_inser)
            #print(percentage)
            if chrom_ref==chrom_inser and ref_pos>=inser_pos-size_inser and ref_pos<=inser_pos+size_inser :
                #print(elt)
                black_list.append(head)
            dic[head].append(percentage)
            #if head in dic_occurrence and size_inser>=1000 and head not in liste_segm_dup and dic_occurrence[head]>=1 and dic_occurrence[head]<=50 :
            #    liste_segm_dup.append(head)
            #    count_segm+=1
            iden=elt[9]
            size_align=elt[18].split(",")
            size_inse=elt[19].split(",")
            for i in range (0,len(size_align)-1):
                couple=(int(size_inse[i]),int(size_inse[i])+int(size_align[i]))
                dic_tes[iden].append(couple)
    for elt in dic :
        #print(elt)
        best=max(dic[elt])
        if elt not in black_list and elt not in liste_segm_dup and dic_occurrence[elt]<=50 and best >=threshold :
                dic_last[elt]=best
                seen_validate.add(elt)
                count+=1
        if elt in black_list and best >=threshold  and elt not in liste_segm_dup and best >=threshold :
            dic_td[elt]=best
            #seen_validate.add(elt)
            count_td+=1
        if elt in liste_segm_dup and best >=threshold :
            dic_segm[elt]=best
            
        if elt not in liste_segm_dup  and best >=threshold :
            dic_not_valid[elt]=best
                #print(best,elt)
            #print(best)
        #break
        #break
    #for e in black_list :
    #    if e in seen_validate :
    #        seen_validate.remove(e)
    #        count-=1
    #print(count,count_td,count_segm,len(dic_not_valid))
    return(dic_last,dic_td,dic_segm,dic_not_valid,dic_tes)

def get_dic_composite(dic_tes):
    dic_size_align=defaultdict()
    for elt in dic_tes :
        #print("before",dic_tes[elt])
        work_list=sorted(dic_tes[elt], key=lambda tup:tup[0]) 
        #print("after",work_list)
        size=0
        st=0
        for elts in work_list :
            if st==0 :
                begin=elts[0]
                end=elts[1]
                st+=1
                continue
            else :
                if elts[1]>end and elts[0]<=end :
                    end=elts[1]
                elif elts[1]<=end or (elts[0]==begin and elts[1]==end):
                    continue
                elif elts[0]>end :
                    size+=end-begin
                    begin=elts[0]
                    end=elts[1]
                else :
                    print("probleme",elts,begin,end)
        size+=end-begin
        dic_size_align[elt]=size
    return dic_size_align

In [ ]:
def make_db(info_file,threshold):
    TRF=csv.reader(open(info_file,"r"),delimiter=" ")
    seen_validate=set()
    dic=defaultdict(list)
    #next(TRF)
    count=0
    dic_last=defaultdict(float)
    dic_seed=defaultdict(list)
    dic_seed_unique=defaultdict(list)
    for elt in TRF :
        #print(elt)
        if len(elt[0].split("_"))<=4 :
            #print(elt)
            size_trf =check_size_align(int(elt[2]),int(elt[1]))+1
            parser=elt[0].split("_")
            chromosome=re.sub("chr", "",parser[0])
            position=int(parser[-2])
            size=int(parser[-1])
            percentage=float(size_trf)/float(size)
            head=chromosome+"_"+str(position)+"_"+str(size)
            #print (percentage)
            if percentage >=threshold :
                dic[head].append(percentage)
                compound=(float(elt[3]),float(elt[4]))
                dic_seed[head].append(compound)
            
    for elt in dic :
        #print(dic[elt],elt)
        best=max(dic[elt])
        if best >=threshold :
            dic_last[elt]=best
            seen_validate.add(elt)
            count+=1
            #print(best)
    for elt in dic_seed :
        if len(dic_seed[elt])!=1 :
            lower_seed=dic_seed[elt][0][0]
            max_rep=dic_seed[elt][0][1]
            for e in dic_seed[elt]:
                if e[0]< lower_seed and e[1]>max_rep :
                    lower_seed=e[0]
                    max_rep=e[1]
            dic_seed_unique[elt]=(lower_seed,max_rep)
        else :
            dic_seed_unique[elt]=(dic_seed[elt][0][0],dic_seed[elt][0][1])
    print(count)
    return(dic_last,dic_seed_unique)

    
    

In [ ]:
def get_validate_variant(vcf,dictionary,otp,otp_2) :
    for elt in vcf :
        #print(elt)
        if "#" not in elt[0] and "@" not in elt[0] :
            chrom = re.sub("chr", "", elt[0])
            pos=int(elt[1])
            if chrom in dictionary and pos in dictionary[chrom] :
                otp.writerow(elt)
            else :
                otp_2.writerow(elt)

In [ ]:
def make_db_me_other(info_file,threshold,delimiters):
    seen_validate=set()
    dic=defaultdict(list)
    seen=[]
    #next(info_file)
    ME=csv.reader(open(info_file,"r"),delimiter=delimiters)
    count=0
    liste_found=[]
    dic_last=defaultdict(float)
    for elt in ME :
        if "#" not in elt[0] :
            clean_list = [x for x in elt if x != '']
            parser=clean_list[2].split("_")
            if len(parser)<=4 :
                #print(clean_list)
                size_aligned =check_size_align(int(clean_list[12]),int(clean_list[11]))+1
                parser=clean_list[2].split("_")
                chromosome=re.sub("chr", "",parser[0])
                position=int(parser[1])
                size=int(parser[2])
                #print(clean_list)
                #print(size_aligned,size)
                percentage=float(size_aligned)/float(size)
                #print(size,size_aligned,percentage)
                #print(clean_list)
                head=chromosome+"_"+str(position)+"_"+str(size)
                if percentage >=threshold :
                    dic[head].append(percentage)

    for elt in dic :
        #print(dic[elt],elt)
        best=max(dic[elt])
        if best >=threshold :
            dic_last[elt]=best
            seen_validate.add(elt)
            count+=1
            #print(dic[elt],elt,best)
    print(count)
    return(dic_last)
def make_db_me(info_file,threshold,delimiters):
    seen_validate=set()
    dic=defaultdict(list)
    seen=[]
    #next(info_file)
    ME=csv.reader(open(info_file,"r"),delimiter=delimiters)
    count=0
    liste_found=[]
    dic_last=defaultdict(float)
    for elt in ME :
        if "#" not in elt[0] :
            clean_list = [x for x in elt if x != '']
            parser=clean_list[2].split("_")
            if len(parser)<=4 :
                #print(clean_list)
                size_aligned =check_size_align(int(clean_list[12]),int(clean_list[11]))+1
                parser=clean_list[2].split("_")
                chromosome=re.sub("chr", "",parser[0])
                position=int(parser[-2])
                size=int(parser[-1])
                #print(clean_list)
                #print(size_aligned,size)
                percentage=float(size_aligned)/float(size)
                #print(size,size_aligned,percentage)
                #print(clean_list)
                head=chromosome+"_"+str(position)+"_"+str(size)
                if percentage >=threshold :
                    dic[head].append(percentage)

    for elt in dic :
        #print(dic[elt],elt)
        best=max(dic[elt])
        if best >=threshold :
            dic_last[elt]=best
            seen_validate.add(elt)
            count+=1
            #print(dic[elt],elt,best)
    print(count)
    return(dic_last)
def count_same(liste_t,liste_q):
    count=0
    liste_unfound=[]
    for elt in liste_q :
        if elt in liste_t :
            count+=1
        else :
            liste_unfound.append(elt)
    if len(liste_unfound)==0 or len(liste_q)==0 :
        FP=0
    else :
        FP=len(liste_unfound)/len(liste_q)
    print("found ",count,",percentage :",count/len(liste_t),",FP :",FP,",len ref",len(liste_t),",len query",len(liste_q))
    #return(liste_unfound)

In [ ]:
def get_header(el) :
    splt=el.split("_")
    if len(splt)==4:
        #print(element)
        header=re.sub("chr", "",splt[0])+"_"+splt[2]+"_"+splt[3]
        #print(header)
    else :
        header=re.sub("chr", "",splt[0])+"_"+splt[1]+"_"+splt[2]
    return header
def increment_dic(dic,liste,type_sv) :
    count=0
    if len(liste)==0 :
        return dic
    for element in liste :
        header=get_header(element)
        if header not in dic :
            #print("ERROR",header)
            continue
        if header in dic and type_sv not in dic[header]:
            dic[header].add(type_sv)
            count+=1
            #print(header,type_sv,dic[header])
        #if type_sv in dic[header] :
            #print (header, dic[header], element)
    print(type_sv,count)
    return dic


In [ ]:
def is_valid(inser):
    allowed = "ATCGatcg"
    if all(c in allowed for c in inser):
        return True
    else:
        return False
def get_dic_all(inpt) :
    dic_all=[]
    inpt_seq=csv.reader(open(inpt,"r"), delimiter="\t")
    for element in inpt_seq :
        if "#" not in element[0] and "@" not in element[0] :
            parser = element[7].split(';')
            svseq=parser[6].split("=")[1].upper()
            chrom = re.sub("chr", "", element[0])
            position=int(element[1])
            headers=chrom+"_"+str(position)+"_"+str(len(svseq))
            if headers not in dic_all and is_valid(svseq) and len(svseq)>=50 :
                dic_all.append(headers)
    return dic_all

def get_dic_all_2(inpt) :
    dic_all={}
    inpt_seq=csv.reader(open(inpt,"r"), delimiter="\t")
    for elt in inpt_seq :
        if "#" not in elt[0] and len(elt[4])>len(elt[3]) and is_valid(elt[4]) :
            headers=re.sub("chr", "",elt[0])+"_"+elt[1] +"_"+str(len(elt[4]))
            if headers not in dic_all :
                dic_all[headers]={"all"}
    return dic_all

def count_novo(dic_all) :
    count_unique=0
    tot_dup=0
    tot_trf=0
    tot_tdup=0
    tot_me=0
    tot_cplx=0
    reverse_dic={}
    for elt in dic_all :
        if tuple(dic_all[elt]) not  in reverse_dic :
            reverse_dic[tuple(dic_all[elt])]=1
        else :
            reverse_dic[tuple(dic_all[elt])]+=1
        if len(dic_all[elt])==1  and "all" in dic_all[elt]:
            #print(dic_all[elt], elt)
            count_unique+=1
            continue
        elif "ME" in dic_all[elt] :
            tot_me+=1
            continue
        elif "TRF" in dic_all[elt] :
            tot_trf+=1
            continue
        elif "TD" in dic_all[elt] :
            tot_tdup+=1
            continue
        elif "DUP" in dic_all[elt] :
            tot_dup+=1
            continue
        else :
            tot_cplx+=1
    print("new : ",count_unique, " cplx : ", tot_cplx, " remove : TD :", tot_tdup, " TRF : ",tot_trf, " DUP : ",tot_dup, " ME : ",tot_me)
    return (reverse_dic)


In [ ]:

def normalized_size (dic,incr):
    for element in dic :
        if len(dic[element])<incr :
            dic[element].append(0)
    return dic

def get_perfec_dic(incr,dic_alone,dic_merge) :
    for elt in dic_alone :
        if len(elt)>2:
            if elt in dic_merge :
                dic_merge[elt].append(int(dic_alone[elt]))
            else :
                if incr==1:
                    dic_merge[elt]=[int(dic_alone[elt])]
                else :
                    dic_merge[elt]=[int(dic_alone[elt])]
                while len(dic_merge[elt])<incr :
                    dic_merge[elt].append(0)
                
    dic_merge=normalized_size(dic_merge,incr)
    return dic_merge


In [ ]:
def compare(td,tr,me,dup,segm) :
    liste_tot=[td,tr,me,dup,segm]
    liste_tot.sort(key = lambda x: x[1],reverse=True)
    #print(liste_tot)
    return liste_tot[0]
def count_v2(dic_all,dic_td,dic_tr,dic_me,dic_dup,threshold):
    count_novo=0
    count_me=0
    count_tr=0
    count_td=0
    count_dup=0
    count_order_me=0
    count_order_tr=0
    count_order_td=0
    count_order_dup=0
    liste_novo=[]
    liste_td=[]
    liste_me=[]
    liste_tr=[]
    liste_dup=[]
    liste_td_o=[]
    liste_me_o=[]
    liste_tr_o=[]
    liste_dup_o=[]
    for elt in dic_all :
        #print(elt,dic_all[elt])
        #print(dic_td,dic_td[elt],dic_tr,dic_tr[elt],dic_me,dic_me[elt],dic_dup,dic_dup[elt])
        liste_to_compare=[]
        best_td=["TD",0]
        best_tr=["TR",0]
        best_me=["ME",0]
        best_dup=["DUP",0]
        if elt in dic_td :
            best_td=["TD",dic_td[elt]]
        if elt in dic_tr :
            best_tr=["TR",dic_tr[elt]]
        if elt in dic_me :
            best_me=["ME",dic_me[elt]]
        if elt in dic_dup :
            best_dup=["DUP",dic_dup[elt]]
        if elt not in dic_td and elt not in dic_tr and elt not in dic_me and elt not in dic_dup :
            #print(elt)
            count_novo+=1
            liste_novo.append(dic_all[elt])
            continue
        #if threshold!=1 :
        #    if best_td[1]<1-threshold and best_tr[1]<1-threshold and best_me[1]<1-threshold and best_dup[1]<1-threshold :
        #        count_novo+=1
        best=compare(best_td,best_tr,best_me,best_dup)
        if threshold !=0 :
            if best[1]>=threshold :
                if best[0]=="TD"  :
                    count_td+=1
                    liste_td.append(dic_all[elt])
                elif best[0]=="TR" :
                    count_tr+=1
                    liste_tr.append(dic_all[elt])
                elif best[0]=="ME"  :
                    count_me+=1
                    liste_me.append(dic_all[elt])
                elif best[0]=="DUP"  :
                    count_dup+=1
                    liste_dup.append(dic_all[elt])
        else :
            if best[0]=="TD"  :
                count_td+=1
                liste_td.append(dic_all[elt])
            elif best[0]=="TR" :
                count_tr+=1
                liste_tr.append(dic_all[elt])
            elif best[0]=="ME"  :
                count_me+=1
                liste_me.append(dic_all[elt])
            elif best[0]=="DUP"  :
                count_dup+=1
                liste_dup.append(dic_all[elt])

        if elt in dic_me :
            count_order_me+=1
            liste_me_o.append(dic_all[elt])
            continue
        if elt in dic_tr :
            count_order_tr+=1
            liste_tr_o.append(dic_all[elt])
            continue
        if elt in dic_td :
            count_order_td+=1
            liste_td_o.append(dic_all[elt])
            continue
        if elt in dic_dup :
            count_order_dup+=1
            liste_dup_o.append(dic_all[elt])
            continue
        #break
    print ("BEST align ","novo : ", count_novo, "TD : ", count_td, "TR : ", count_tr, "ME : ", count_me, "DUP : ", count_dup)
    total=count_novo+count_td+count_tr+count_me+count_dup
    total_order=count_novo+count_order_td+count_order_tr+count_order_me+count_order_dup
    print ("Order ","novo : ", count_novo, "TD : ", count_order_td, "TR : ", count_order_tr, "ME : ", count_order_me, "DUP : ", count_order_dup)
    print("TOTAL :", total, " TOTAL order : ",total_order)
    #return liste_td,liste_me,liste_tr,liste_dup,liste_novo,liste_td_o,liste_me_o,liste_tr_o,liste_dup_o

In [ ]:
def get_list(dic_all,dic_td,dic_tr,dic_me,dic_dup,segm_dup,not_valid,threshold,compo):
    count_novo=0
    count_novo_2=0
    count_me=0
    count_tr=0
    count_td=0
    count_dup=0
    count_segm=0
    count_order_me=0
    count_order_tr=0
    count_order_td=0
    count_order_dup=0
    count_order_segm=0
    liste_novo=[]
    liste_td=[]
    liste_me=[]
    liste_tr=[]
    liste_dup=[]
    liste_segm=[]
    liste_td_o=[]
    liste_me_o=[]
    liste_tr_o=[]
    liste_dup_o=[]
    liste_segm_o=[]
    print(len(segm_dup))
    for elt in dic_all :
        if elt not in dic_td and elt not in segm_dup and elt not in dic_tr and elt not in dic_me and elt not in dic_dup and elt not in segm_dup and elt not in not_valid :
            #print(elt)
            count_novo+=1
            liste_novo.append(elt)
            #if elt not in compo :
            #    count_novo+=1
            #    liste_novo.append(elt)
            #elif compo[elt]/int(elt.split("_")[-1])<threshold :
            #    count_novo_2+=1
            #    liste_novo.append(elt)
            #else :
            #    print(elt,compo[elt])
        #if threshold!=1 :
        #    if best_td[1]<1-threshold and best_tr[1]<1-threshold and best_me[1]<1-threshold and best_dup[1]<1-threshold :
        #        count_novo+=1
        if elt in dic_me :
            count_order_me+=1
            liste_me_o.append(elt)
            continue
        if elt in dic_tr :
            count_order_tr+=1
            liste_tr_o.append(elt)
            continue
        if elt in segm_dup :
            count_order_segm+=1
            liste_segm_o.append(elt)
            continue
        if elt in dic_td :
            count_order_td+=1
            liste_td_o.append(elt)
            continue
        if elt in dic_dup :
            count_order_dup+=1
            liste_dup_o.append(elt)
            continue
        #break
    total_order=count_novo+count_order_td+count_order_tr+count_order_me+count_order_dup
    print ("Order ","novo : ", count_novo, "TD : ", count_order_td, "TR : ", count_order_tr, "ME : ", count_order_me, "DUP : ", count_order_dup)#, "SEGM : ",count_order_segm)
    #print(" TOTAL order : ",total_order)
    return liste_td,liste_me,liste_tr,liste_dup,liste_novo,liste_td_o,liste_me_o,liste_tr_o,liste_dup_o,liste_segm_o

In [ ]:
def get_complex(dic,td,tr,me,dup,novo,segm):
    count=0
    liste_complex=[]
    for head in dic :
        if head not in novo and head not in td and head not in me and head not in tr and head not in dup and head not in segm:
            count+=1
            liste_complex.append(head)
    print(count)
    return liste_complex

In [ ]:

def get_occu(liste,threshold):
    count=0
    for i in liste :
        if i>= threshold :
            count+=1
    return count
liste_chais=[]
inpt=csv.reader(open("/home/wesley/genouest/Blast/Chaisson/ME/Mobile_Elt_Chaisson_HG38.vcf"),delimiter="\t")
for element in inpt :
    parser = element[7].split(';')
    svseq=parser[6].split("=")[1].upper()
    chrom = re.sub("chr", "", element[0])
    position=int(element[1])
    headers=chrom+"_"+str(position)+"_"+str(len(svseq))
    liste_chais.append(headers)
def get_list_occu(info_file,threshold,dic_all): 
    inpt=csv.reader(open(info_file,"r"),delimiter="\t")
    count=0
    liste_all=defaultdict()
    dic=defaultdict(list)

    count_td=0
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    for elt in inpt :
        if len(elt[9].split("_"))<=4 :
            match=int(elt[0])
            size_align=check_size_align(int(elt[12]),int(elt[11]))
            size_align_ref=check_size_align(int(elt[16]),int(elt[15]))
            size_inser=int(elt[9].split("_")[-1])
            chrom_inser=re.sub("chr", "",elt[9].split("_")[0])
            inser_pos=int(elt[9].split("_")[2])
            chrom_ref=re.sub("chr", "",elt[13])
            ref_pos=int(elt[15])
            heads=chrom_inser+"_"+str(inser_pos)+"_"+str(size_inser)
            #print(size_align,size_inser,chrom_inser,inser_pos,chrom_ref,ref_pos)
            percentage=float(size_align/size_inser)
            percentage_match=float(match/size_inser)
            dic[heads].append(percentage)
    for head in dic :
        if head in dic_all :
            counts=get_occu(dic[head],threshold)
            liste_all[head]=counts
        else :
            liste_all[head]=0
    return liste_all
def get_list_occu_2(info_file,threshold,dic_all): 
    inpt=csv.reader(open(info_file,"r"),delimiter="\t")
    count=0
    liste_all=defaultdict()
    dic=defaultdict(list)

    count_td=0
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    for elt in inpt :
        match=int(elt[0])
        size_align=check_size_align(int(elt[12]),int(elt[11]))
        size_align_ref=check_size_align(int(elt[16]),int(elt[15]))
        size_inser=int(elt[9].split("_")[-1])
        chrom_inser=re.sub("chr", "",elt[9].split("_")[0])
        inser_pos=int(elt[9].split("_")[1])
        chrom_ref=re.sub("chr", "",elt[13])
        ref_pos=int(elt[15])
        heads=chrom_inser+"_"+str(inser_pos)+"_"+str(size_inser)
        #print(size_align,size_inser,chrom_inser,inser_pos,chrom_ref,ref_pos)
        percentage=float(size_align/size_inser)
        percentage_match=float(match/size_inser)
        dic[heads].append(percentage)
    for head in dic :
        if head in dic_all :
            counts=get_occu(dic[head],threshold)
            liste_all[head]=counts
        else :
            liste_all[head]=0
    return liste_all
def get_list_percentage(info_file,threshold,td,me,tr,dup,novo,td_o,me_o,tr_o,dup_o,complexe,segm,dic_all): 
    inpt=csv.reader(open(info_file,"r"),delimiter="\t")
    count=0
    seen_validate=set()
    dic=defaultdict(list)
    dic_last=defaultdict(float)
    black_list=[]
    dic_td=defaultdict(float)
    count_td=0
    liste_occurence_me=[]
    liste_occurence_tr=[]
    liste_occurence_td=[]
    liste_occurence_dp=[]
    liste_occurence_novo=[]
    liste_occurence_me_o=[]
    liste_occurence_tr_o=[]
    liste_occurence_td_o=[]
    liste_occurence_dp_o=[]
    liste_count_size=[]
    liste_low_size=[]
    liste_occurrence_complex=[]
    liste_unassigned=[]
    liste_all=defaultdict()
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    for elt in inpt :
        match=int(elt[0])
        size_align=check_size_align(int(elt[12]),int(elt[11]))
        size_align_ref=check_size_align(int(elt[16]),int(elt[15]))
        size_inser=int(elt[9].split("_")[3])
        chrom_inser=re.sub("chr", "",elt[9].split("_")[0])
        inser_pos=int(elt[9].split("_")[2])
        chrom_ref=re.sub("chr", "",elt[13])
        ref_pos=int(elt[15])
        heads=chrom_inser+"_"+str(inser_pos)+"_"+str(size_inser)
        #print(size_align,size_inser,chrom_inser,inser_pos,chrom_ref,ref_pos)
        percentage=float(size_align/size_inser)
        percentage_match=float(match/size_inser)
        dic[heads].append(percentage)
    for he in novo :
        if he not in dic :
            liste_occurence_novo.append(0)
            liste_low_size.append(0)
    for head in dic :
        if head in dic_all :
            counts=get_occu(dic[head],threshold)
            if counts >= 600 :
                liste_count_size.append(counts)
            if counts < 600 :
                liste_low_size.append(counts)
            if head in td :
                liste_occurence_td.append(counts)
            if head in me :
                liste_occurence_me.append(counts)
            if head in tr :
                liste_occurence_tr.append(counts)
            if head in dup :
                liste_occurence_dp.append(counts)
            if head in novo :
                liste_occurence_novo.append(counts)
            if head in td_o :
                liste_occurence_td_o.append(counts)
            if head in me_o :
                liste_occurence_me_o.append(counts)
            if head in tr_o :
                liste_occurence_tr_o.append(counts)
            if head in dup_o :
                liste_occurence_dp_o.append(counts)
            if head in complexe :
                liste_occurrence_complex.append(counts)
            if head in segm :
                liste_unassigned.append(counts)
            liste_all[head]=counts
    return liste_occurence_novo,liste_occurence_td_o,liste_occurence_me_o,liste_occurence_tr_o,liste_occurence_dp_o,liste_occurrence_complex,liste_unassigned,liste_all


In [ ]:
def get_list_percentage_size(info_file,threshold,td,me,tr,dup,novo,td_o,me_o,tr_o,dup_o,complexe,dic_lim): 
    inpt=csv.reader(open(info_file,"r"),delimiter="\t")
    count=0
    seen_validate=set()
    dic=defaultdict(list)
    dic_last=defaultdict(float)
    black_list=[]
    dic_td=defaultdict(float)
    count_td=0
    liste_occurence_me=[]
    liste_occurence_tr=[]
    liste_occurence_td=[]
    liste_occurence_dp=[]
    liste_occurence_novo=[]
    liste_occurence_me_o=[]
    liste_occurence_tr_o=[]
    liste_occurence_td_o=[]
    liste_occurence_dp_o=[]
    liste_occurence_novo_o=[]
    liste_count_size=[]
    liste_low_size=[]
    low_chaiss=[]
    high_chaiss=[]
    liste_occurence_complexe_low=[]
    liste_occurence_complexe_high=[]
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    for elt in inpt :
        match=int(elt[0])
        size_align=check_size_align(int(elt[12]),int(elt[11]))
        size_align_ref=check_size_align(int(elt[16]),int(elt[15]))
        size_inser=int(elt[9].split("_")[3])
        chrom_inser=re.sub("chr", "",elt[9].split("_")[0])
        inser_pos=int(elt[9].split("_")[2])
        chrom_ref=re.sub("chr", "",elt[13])
        ref_pos=int(elt[15])
        heads=chrom_inser+"_"+str(inser_pos)+"_"+str(size_inser)
        #print(size_align,size_inser,chrom_inser,inser_pos,chrom_ref,ref_pos)
        percentage=float(size_align/size_inser)
        percentage_match=float(match/size_inser)
        dic[heads].append(percentage)
    for he in novo :
        if he not in dic :
            liste_occurence_novo_o.append(int(he.split("_")[2]))
    for head in dic :
        counts=get_occu(dic[head],threshold)
        if head in dic_lim :
            if int(head.split("_")[2])==0:
                        print(head)
            if counts >= 600 :
                liste_count_size.append(int(head.split("_")[2]))
                #if head in chaisson_liste :
                #    high_chaiss.append(int(head.split("_")[2]))
                if head in td_o :
                    liste_occurence_td.append(int(head.split("_")[2]))
                if head in me_o :
                    liste_occurence_me.append(int(head.split("_")[2]))
                if head in tr_o :
                    liste_occurence_tr.append(int(head.split("_")[2]))
                if head in dup :
                    #print(head)
                    liste_occurence_dp.append(int(head.split("_")[2]))
                if head in novo :
                    liste_occurence_novo.append(int(head.split("_")[2]))
                if head in complexe :
                    liste_occurence_complexe_high.append(int(head.split("_")[2]))
            if counts < 600 :
                liste_low_size.append(int(head.split("_")[2]))
                #if head in chaisson_liste :
                #    low_chaiss.append(int(head.split("_")[2]))
                if head in novo :
                    liste_occurence_novo_o.append(int(head.split("_")[2]))
                if head in td_o :
                    liste_occurence_td_o.append(int(head.split("_")[2]))
                if head in me_o :
                    liste_occurence_me_o.append(int(head.split("_")[2]))
                if head in tr_o :
                    liste_occurence_tr_o.append(int(head.split("_")[2]))
                if head in dup_o :
                    liste_occurence_dp_o.append(int(head.split("_")[2]))
                if head in complexe :
                    liste_occurence_complexe_low.append(int(head.split("_")[2]))
    return liste_occurence_td,liste_occurence_me,liste_occurence_tr,liste_occurence_dp,liste_occurence_novo,liste_occurence_td_o,liste_occurence_me_o,liste_occurence_tr_o,liste_occurence_dp_o,liste_occurence_novo_o,liste_count_size,liste_low_size,low_chaiss,high_chaiss,liste_occurence_complexe_high,liste_occurence_complexe_low


In [ ]:
def get_list_head(info_file,threshold,td,me,tr,dup,novo,td_o,me_o,tr_o,dup_o,complexe,dic_lim): 
    inpt=csv.reader(open(info_file,"r"),delimiter="\t")
    count=0
    seen_validate=set()
    dic=defaultdict(list)
    dic_last=defaultdict(float)
    black_list=[]
    dic_td=defaultdict(float)
    count_td=0
    liste_occurence_me=[]
    liste_occurence_tr=[]
    liste_occurence_td=[]
    liste_occurence_dp=[]
    liste_occurence_novo=[]
    liste_occurence_me_o=[]
    liste_occurence_tr_o=[]
    liste_occurence_td_o=[]
    liste_occurence_dp_o=[]
    liste_count_size=[]
    liste_low_size=[]
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    next(inpt)
    for elt in inpt :
        match=int(elt[0])
        size_align=check_size_align(int(elt[12]),int(elt[11]))
        size_align_ref=check_size_align(int(elt[16]),int(elt[15]))
        size_inser=int(elt[9].split("_")[3])
        chrom_inser=re.sub("chr", "",elt[9].split("_")[0])
        inser_pos=int(elt[9].split("_")[2])
        chrom_ref=re.sub("chr", "",elt[13])
        ref_pos=int(elt[15])
        heads=chrom_inser+"_"+str(inser_pos)+"_"+str(size_inser)
        #print(size_align,size_inser,chrom_inser,inser_pos,chrom_ref,ref_pos)
        percentage=float(size_align/size_inser)
        percentage_match=float(match/size_inser)
        dic[heads].append(percentage)
    for he in novo :
        if he in dic_lim :
            if he not in dic :
                liste_low_size.append(he)
    for head in dic :
        if head in dic_lim :
            counts=get_occu(dic[head],threshold)
            if counts >= 600 :
                liste_count_size.append(head)
            if counts < 600 :
                liste_low_size.append(head)
       
    return liste_count_size,liste_low_size


In [3]:
def convert_plus_minus(start,end):
    if start<=end :
        return start,end
    else :
        return end,start
def check_strand(a,b):
    if int(b)<int(a) :
        return (a-b)+1
    else :
        return (b-a)+1
def check_validity(ide,pos,size_ins):
    if ide =="start" and pos>=size_ins-10 :
        return True
    if ide =="end" and pos<=10 :
        return True

def check_homology(couple) :
    homo_start=False
    homo_end=False
    if couple[0] =="start" :
        valid_query=check_validity(couple[0],couple[3],couple[-1])
        if valid_query :
            homo_start=check_validity("end",couple[5],couple[-1])
            homo_end=check_validity("start",couple[6],couple[-1])
    if couple[0] =="end" :
        valid_query=check_validity(couple[0],couple[2],couple[-1])
        if valid_query :
            homo_start=check_validity("end",couple[5],couple[-1])
            homo_end=check_validity("start",couple[6],couple[-1])
            
    if homo_start and homo_end :
        return 1,1
    if homo_start and not homo_end :
        return 1,0
    if not homo_start and homo_end :
        return 0,1
    else :
        return 0,0
def get_couple(elt):
    ref_identifiant=elt[0].split("_")[-1]
    ref_size_insertion=int(elt[1].split("_")[-1])
    ref_qs=int(elt[6])
    ref_qe=int(elt[7])
    ref_ss=int(elt[8])
    ref_se=int(elt[9])
    ref_size_align_query=int(check_strand(ref_qs,ref_qe))
    ref_size_align_s=int(check_strand(ref_ss,ref_se))
    ref_couple=[ref_identifiant,int(elt[3]),ref_qs,ref_qe,ref_size_align_s,ref_ss,ref_se,ref_size_insertion]
    return ref_couple 

def get_info(elt):
    ref_couple=[]
    ref=elt[0]
    key=re.sub("chr", "",elt[1].split("_")[0])+"_"+elt[1].split("_")[1]+"_"+elt[1].split("_")[-1]
    ref_couple=get_couple(elt)
    homo_start,homo_end=check_homology(ref_couple)
    ref_couple.append(homo_start)
    ref_couple.append(homo_end)
    return ref,key,ref_couple
def get_all(inpt) :
    dic=defaultdict(list)
    inpts=csv.reader(open(inpt,"r"), delimiter='\t')
    ref=""
    ref_couple_end=[]
    ref_couple_start=[]
    for elt in inpts :
        if int(elt[8])< int(elt[9]) and int(elt[6])<int(elt[7]) :
            if int(elt[8])<=10 :
                if ref =="" or not ref_couple_start :
                    ref,key,ref_couple_start=get_info(elt)
                    continue
                elif elt[0] ==ref  :
                    couple=get_couple(elt)
                    homo_start,homo_end=check_homology(couple)
                    couple.append(homo_start)
                    couple.append(homo_end)
                    #print(homo_start,elt[8],ref_couple[5],homo_end,elt[9],ref_couple[6])
                    if (homo_start and int(elt[8])<ref_couple_start[5]):
                        ref_couple_start=couple
                    elif (homo_start and int(elt[8])==ref_couple_start[5] and int(couple[1])>int(ref_couple_start[1])):
                        ref_couple_start=couple
                   # else : 
                    #    print("START",ref_couple_start,couple)
                elif elt[0]!=ref and ref_couple_start :
                    #print(elt,ref_couple)
                    if ref_couple_start[8] or ref_couple_start[9] :
                        dic[key].append(ref_couple_start)
                        #break
                    ref,key,ref_couple_start=get_info(elt)
                    continue

            elif int(elt[9])>=int(elt[1].split("_")[-1])-10 :
                #print(elt)
                if ref =="" or not ref_couple_end:
                    ref,key,ref_couple_end=get_info(elt)
                    continue
                elif elt[0] ==ref  :
                    couple=get_couple(elt)
                    homo_start,homo_end=check_homology(couple)
                    couple.append(homo_start)
                    couple.append(homo_end)
                 #   print("COUPLE",couple,ref_couple_end)
                    #print(homo_start,elt[8],ref_couple[5],homo_end,elt[9],ref_couple[6])
                    if  (homo_end and int(elt[9])>ref_couple_end[6]):
                        ref_couple_end=couple
                    elif (homo_end and int(elt[9])==ref_couple_end[6]and int(couple[1])>int(ref_couple_end[1])):
                        ref_couple_end=couple
                  #  else :
                   #      print("END",ref_couple_end,couple)
                elif elt[0]!=ref and ref_couple_end :
                    #print(elt,ref_couple)
                    if ref_couple_end[8] or ref_couple_end[9] :
                        dic[key].append(ref_couple_end)
                        #break
                    ref,key,ref_couple_end=get_info(elt)
                    continue
            #else :
                #print("PB", elt)
    dic[key].append(ref_couple_start)
    dic[key].append(ref_couple_end)
    #print(dic)s_2
    return dic

def tests (inpt,threshold) :
    dic=get_all(inpt)
    return dic


def get_dic_tandem_homology(d,threshold,liste_tandem):
    tandem_dic_start=defaultdict(list)
    tandem_dic_end=defaultdict(list)
    tandem_dic_both=defaultdict(list)
    to_remove=[]
    for elt in d :
        if elt in liste_tandem :
            size_inser=int(elt.split("_")[-1])
            tandem_start=False 
            tandem_end=False
            for e in d[elt] :
                #print(e)
                if (e[0] =="start" and e[-1]==1 and e[-2]==1) or int(e[4])/size_inser >=threshold :
                    tandem_start=True
                if (e[0] == "end" and e[-1]==1 and e[-2]==1) or int(e[4])/size_inser >=threshold :
                    tandem_end=True
            if tandem_start and not tandem_end :
                tandem_dic_start[elt]=d[elt]
            if not tandem_start and  tandem_end :
                tandem_dic_end[elt]=d[elt]
            if tandem_start and  tandem_end :
                #print(elt)
                tandem_dic_both[elt]=d[elt]

            if tandem_start  or tandem_end :
                to_remove.append(elt)
    for i in to_remove :   
        if i in d :
            del d[i]
    return d,tandem_dic_start,tandem_dic_end,tandem_dic_both

In [1]:
def get_all_2s(inpt,d,t) :
    dic=defaultdict(list)
    inpts=csv.reader(open(inpt,"r"), delimiter='\t')
    for elt in inpts :
        #print(elt)
        key=re.sub("chr", "",elt[1].split("_")[0])+"_"+elt[1].split("_")[1]+"_"+elt[1].split("_")[-1]
        ide_perc=float(int(elt[3])-int(elt[4])-int(elt[5]))/int(elt[3])
        if int(elt[9])>int(elt[8]) and int(elt[7])>int(elt[6]) and ide_perc>=t and (int(elt[9])>=int(elt[1].split("_")[-1])-10 or int(elt[8])<=10) :
            ref_couple=get_couple(elt)
            homo_start,homo_end=check_homology(ref_couple)
            
            ref_couple.append(homo_start)
            ref_couple.append(homo_end)
            dic[key].append(ref_couple)
    return dic
def process_2 (dic) :
    dic_final=defaultdict(list)
    for elt in dic :
        i=0
        if len(dic[elt])==1 :
            dic_final[elt]=dic[elt]
        else :
            for i in range (len(dic[elt])) :
                if i==0 :
                    ref =dic[elt][0]
                else :
                    if (dic[elt][i][-2] and int(dic[elt][i][1])>int(ref[1])) or (dic[elt][i][-1] and int(dic[elt][i][1])>int(ref[1])):
                        ref=dic[elt][i]
            dic_final[elt]=ref
    return dic_final
def tests_v2 (inpt,threshold,d,t) :
    dic=get_all_2s(inpt,d,t)
    dic_f=process_2(dic)
    return dic_f

SyntaxError: invalid syntax (<ipython-input-1-88c9ee537311>, line 19)

In [4]:
def get_all_2(inpt,d,t) :
    dic=defaultdict(list)
    inpts=csv.reader(open(inpt,"r"), delimiter='\t')
    ref=""
    ref_couple=[]
    couple=[]
    for elt in inpts :
        old_key=re.sub("chr", "",elt[1].split("_")[0])+"_"+elt[1].split("_")[1]+"_"+elt[1].split("_")[-1]
        if old_key=="3_88522029_341" :
            print(elt)
        ide_perc=float(int(elt[3])-int(elt[4])-int(elt[5]))/int(elt[3])
        #print(elt,ide_perc)
        if int(elt[9])>int(elt[8]) and int(elt[7])>int(elt[6]) and ide_perc>=t : #and key in d :
            if int(elt[9])>=int(elt[1].split("_")[-1])-10 or int(elt[8])<=10 or int(elt[3])!=0 :
                if ref =="":
                    ref=elt[0]
                    key=re.sub("chr", "",elt[1].split("_")[0])+"_"+elt[1].split("_")[1]+"_"+elt[1].split("_")[-1]
                    ref_couple=get_couple(elt)
                    homo_start,homo_end=check_homology(ref_couple)
                    ref_couple.append(homo_start)
                    ref_couple.append(homo_end)
                    if key=="3_88522029_341" :
                        print(elt,ref_couple)
                    continue            
                elif elt[0] ==ref  :
                    couple=get_couple(elt)
                    homo_start,homo_end=check_homology(couple)
                    couple.append(homo_start)
                    couple.append(homo_end)
                    
                    #print(homo_start,elt[8],ref_couple[5],homo_end,elt[9],ref_couple[6])
                    if (homo_start and int(couple[1])>int(ref_couple[1])) or (homo_end and int(couple[1])>int(ref_couple[1])):
                        ref_couple=couple
                elif elt[0]!=ref and ref_couple :
                    #print(elt,"COUPLE",ref_couple)
                    if ref_couple[8] or ref_couple[9] :
                        dic[key].append(ref_couple)
                        if key=="3_88522029_341" :
                            print(ref_couple)
                        #break
                    ref=elt[0]
                    key=re.sub("chr", "",elt[1].split("_")[0])+"_"+elt[1].split("_")[1]+"_"+elt[1].split("_")[-1]
                    ref_couple=get_couple(elt)
                    homo_start,homo_end=check_homology(ref_couple)
                    ref_couple.append(homo_start)
                    ref_couple.append(homo_end)
                    continue
            else :
                print(elt)
        else :
            if elt[0]!=ref and ref_couple :
                    #print(elt,"COUPLE",ref_couple)
                    if ref_couple[8] or ref_couple[9] :
                        dic[key].append(ref_couple)
                        #break
                    if key=="3_88522029_341" :
                        print(ref_couple)
                    ref=elt[0]
                    old_key=re.sub("chr", "",elt[1].split("_")[0])+"_"+elt[1].split("_")[1]+"_"+elt[1].split("_")[-1]
                    ref_couple=get_couple(elt)
                    homo_start,homo_end=check_homology(ref_couple)
                    ref_couple.append(homo_start)
                    ref_couple.append(homo_end)
                    continue
    if ref_couple and (ref_couple[8] or ref_couple[9]) :
                dic[key].append(ref_couple)
    #print(dic)
    return dic

def tests_2 (inpt,threshold,d,t) :
    dic=get_all_2(inpt,d,t)
    return dic


In [ ]:

def split_dic(dic,td,tr,me,novo,dp,cplx) :
    dic_mh_me=defaultdict(list)
    dic_mh_tr=defaultdict(list)
    dic_mh_td=defaultdict(list)
    dic_mh_dp=defaultdict(list)
    dic_mh_novo=defaultdict(list)
    dic_mh_cplx=defaultdict(list)
    for elt in dic :
        if elt in td :
            dic_mh_td[elt]=dic[elt]
        elif elt in tr :
            dic_mh_tr[elt]=dic[elt]
        elif elt in dp :
            dic_mh_dp[elt]=dic[elt]
        elif elt in me :
            dic_mh_me[elt]=dic[elt]
        elif elt in novo :
            dic_mh_novo[elt]=dic[elt]
        elif elt in cplx :
            dic_mh_cplx[elt]=dic[elt]
    #print(len(dic),len(dic_mh_me)+len(dic_mh_tr)+len(dic_mh_td)+len(dic_mh_dp)+len(dic_mh_novo)+len(dic_mh_cplx))
    return dic_mh_me,dic_mh_tr,dic_mh_td,dic_mh_dp,dic_mh_novo,dic_mh_cplx
def get_liste_mh(d):
    liste_match_all_s=[]
    liste_match_left_bs=[]
    liste_match_right_bs=[]
    for elt in d :
        if len(d[elt])==1 :
            couple=d[elt]
            liste_match_all_s.append(int(couple[0][4]))

        elif len(d[elt])==2 :
            couple=d[elt]
            liste_match_left_bs.append(int(couple[0][4]))
            liste_match_right_bs.append(int(couple[1][4]))
        else :
            liste_match_all_s.append(0)
    return liste_match_all_s,liste_match_left_bs,liste_match_right_bs

In [ ]:
def count_cross(d_all):
    count_cross=0
    count_not_cross=0
    count_double_cross=0
    count_double_not_cross=0
    count_double_one_cross=0
    liste_cross=[]
    liste_not_cross=[]
    liste_double_cross=[]
    liste_double_not_cross=[]
    liste_double_one_cross=[]
    tot_not_cross=[]
    for elt in d_all :
        if len(d_all[elt])==1 :
            #print(d[elt])
            if d_all[elt][0][0] =="start" :
                if d_all[elt][0][9]==1 :
                    count_cross+=1
                    liste_cross.append(d_all[elt][0][1])
                else :
                    count_not_cross+=1
                    #print(elt,d[elt])
                    liste_not_cross.append(d_all[elt][0][1])
            if d_all[elt][0][0] =="end" :
                if d_all[elt][0][8]==1 :
                    count_cross+=1
                    liste_cross.append(d_all[elt][0][1])
                else :
                    count_not_cross+=1 
                    liste_not_cross.append(d_all[elt][0][1])
        if len(d_all[elt])==2 :
            if d_all[elt][0][9]==1 and d_all[elt][1][8]==1 :
                #print("cross",d[elt])
                liste_double_cross.append(d_all[elt][0][1]+d_all[elt][1][1])
                count_double_cross+=1
            else :
                liste_double_not_cross.append(d_all[elt][0][1])
                liste_double_not_cross.append(d_all[elt][1][1])
                count_double_not_cross+=1
                #print(d_all[elt])
                tot_not_cross.append(elt)
        
    print(count_cross,count_not_cross,count_double_cross,count_double_not_cross)
    return liste_cross,liste_not_cross,liste_double_cross,liste_double_not_cross

In [1]:
def count_cross_id_old(d_all):
    count_cross=0
    count_not_cross=0
    count_double_cross=0
    count_double_not_cross=0
    count_double_one_cross=0
    liste_cross=defaultdict(list)
    liste_not_cross=defaultdict(list)
    liste_double_cross=defaultdict(list)
    liste_double_not_cross=defaultdict(list)
    liste_double_one_cross=defaultdict(list)
    tot_not_cross=[]
    for elt in d_all :
        if len(d_all[elt])==1 :
            #print(d[elt])
            if d_all[elt][0][0] =="start" :
                if d_all[elt][0][9]==1 :
                    count_cross+=1
                    liste_cross[elt].append(d_all[elt][0][1])
                else :
                    count_not_cross+=1
                    #print(elt,d[elt])
                    liste_not_cross[elt].append(d_all[elt][0][1])
            if d_all[elt][0][0] =="end" :
                if d_all[elt][0][8]==1 :
                    count_cross+=1
                    liste_cross[elt].append(d_all[elt][0][1])
                else :
                    count_not_cross+=1 
                    liste_not_cross[elt].append(d_all[elt][0][1])
        if len(d_all[elt])==2 :
            if d_all[elt][0][9]==1 and d_all[elt][1][8]==1 :
                #print("cross",d[elt])
                liste_double_cross[elt].append(d_all[elt][0][1]+d_all[elt][1][1])
                count_double_cross+=1
            else :
                liste_double_not_cross[elt].append(d_all[elt][0][1])
                liste_double_not_cross[elt].append(d_all[elt][1][1])
                count_double_not_cross+=1
                #print(d_all[elt])
                tot_not_cross.append(elt)
        
    print(count_cross,count_not_cross,count_double_cross,count_double_not_cross)
    return liste_cross,liste_not_cross,liste_double_cross,liste_double_not_cross

In [ ]:
def count_cross_id(d_all):
    count_cross=0
    count_not_cross=0
    count_double_cross=0
    count_double_not_cross=0
    count_double_one_cross=0
    liste_cross=defaultdict(list)
    liste_not_cross=defaultdict(list)
    liste_double_cross=defaultdict(list)
    liste_double_not_cross=defaultdict(list)
    liste_double_one_cross=defaultdict(list)
    tot_not_cross=[]
    for elt in d_all :
        if len(d_all[elt])==1 :
            #print(d[elt])
            if d_all[elt][0][0] =="start" :
                if d_all[elt][0][9]==1 :
                    count_cross+=1
                    liste_cross[elt].append(d_all[elt][0][1])
            if d_all[elt][0][0] =="end" :
                if d_all[elt][0][8]==1 :
                    count_cross+=1
                    liste_cross[elt].append(d_all[elt][0][1])
        if len(d_all[elt])==2 :
            if d_all[elt][0][9]==1 and d_all[elt][1][8]==1 :
                #print("cross",d[elt])
                liste_cross[elt].append(d_all[elt][0][1]+d_all[elt][1][1])
                count_double_cross+=1
        
    return liste_cross

In [ ]:
def get_liste_insertion(inpt):
    lec=csv.reader(open(inpt,"r"),delimiter="\t")
    count=0
    liste=[]
    for elt in lec:
        if "#" not in elt[0] and "@" not in elt[0]:
            chrom_inser=re.sub("chr", "",elt[0])
            if is_valid(elt[4]) :
                svseq=elt[4]
            else :
                parser = elt[7].split(';')
                svseq=parser[6].split("=")[1].upper()
            pos=int(elt[1])
            head=chrom_inser+"_"+str(pos)#+"_"+str(len(svseq))
            liste.append(head)
    return liste

In [ ]:
def get_info(liste,inpt,dic) :
    liste_info=set() 
    lec=csv.reader(open(inpt,"r"),delimiter=",")
    for elt in lec :
        heads=re.sub(r'[ \(\) \' ]', "",elt[0]).split(",")
        #print(elt,heads)
        head=heads[0]+"_"+heads[1]
        liste_info=set()
        for i in range (1,len(elt)):
            liste_info.add(elt[i])
        if liste_info and head in liste :
            dic[head]=list(liste_info)
    return dic
def get_other(liste,dic,types) :
    count=0
    for elt in liste :
        if elt not in dic :
            #print(elt)
            dic[elt]=[types]
            count+=1
    print(count)
    return dic

In [ ]:
def get_dic_all_Zook(inpt) :
    dic_all=[]
    inpt_seq=csv.reader(open(inpt,"r"), delimiter="\t")
    for element in inpt_seq :
        if "#" not in element[0] and "@" not in element[0] :
            parser = element[7].split(';')
            svseq=element[4].upper()
            chrom = re.sub("chr", "", element[0])
            position=int(element[1])
            headers=chrom+"_"+str(position)+"_"+str(len(svseq))
            if headers not in dic_all and is_valid(svseq) :
                dic_all.append(headers)
    return dic_all
def get_dic_all_Zook_pass(inpt) :
    dic_all=[]
    inpt_seq=csv.reader(open(inpt,"r"), delimiter="\t")
    for element in inpt_seq :
        if "#" not in element[0] and "@" not in element[0] :
            parser = element[7].split(';')
            svseq=element[4].upper()
            chrom = re.sub("chr", "", element[0])
            position=int(element[1])
            headers=chrom+"_"+str(position)+"_"+str(len(svseq))
            if headers not in dic_all and is_valid(svseq) and element[6]=="PASS" :
                dic_all.append(headers)
    return dic_all

In [ ]:
def get_small_mh(csv_mh) :
    lec=csv.reader(open(csv_mh,'r'),delimiter='\t')
    dic=defaultdict()
    for elt in lec :
        dic[elt[0]]=elt[1]
    return dic

In [1]:
def check_validity_hom(qpos,tpos,asize,size,types):
    if types =="start" :
        if qpos+asize>=size-10 and tpos+asize>=size-10 :
            return True
        else :
            return False
    if types=="end":
        if qpos<=10 and tpos<=10 :
            return True
        else :
            return False
def get_mh_blat(align_file):
    inpts=csv.reader(open(align_file,"r"), delimiter='\t')
    dic=defaultdict(list)
    i=0
    for elt in inpts :
        query=elt[9]
        target=elt[13]
        first_match_size=int(elt[-3].split(",")[0])
        type_q=elt[9].split("_")[-1]
        size_t=int(elt[13].split("_")[-1])
        if len(elt[-3].split(","))>2:
            first_match_size=int(elt[-3].split(",")[0])
            last_match_size=int(elt[-3].split(",")[-2])
            qfirst_match_pos=int(elt[-2].split(",")[0])
            tfirst_match_pos=int(elt[-1].split(",")[0])
            qlast_match_pos=int(elt[-2].split(",")[-2])
            tlast_match_pos=int(elt[-1].split(",")[-2])
            #print(elt)
            #print(first_match_size,last_match_size)
            #print(qfirst_match_pos,qlast_match_pos)
            #print(tfirst_match_pos,tlast_match_pos)
            first=check_validity_hom(qfirst_match_pos,tfirst_match_pos,first_match_size,size_t,type_q)
            last=check_validity_hom(qlast_match_pos,tlast_match_pos,last_match_size,size_t,type_q)
            #print(first,last)
            if type_q=="start" and last:
                dic[target].append((qlast_match_pos,tlast_match_pos,last_match_size,type_q))
            if type_q=="end" and first :
                dic[target].append((qfirst_match_pos,tfirst_match_pos,first_match_size,type_q))
        else :
            first_match_size=int(elt[-3].split(",")[0])
            qfirst_match_pos=int(elt[-2].split(",")[0])
            tfirst_match_pos=int(elt[-1].split(",")[0])
            first=check_validity_hom(qfirst_match_pos,tfirst_match_pos,first_match_size,size_t,type_q)
            if first:
                dic[target].append((qfirst_match_pos,tfirst_match_pos,first_match_size,type_q))
            #print(elt)
            #print(first_match_size)
            #print(qfirst_match_pos)
            #print(tfirst_match_pos)
            #print(first)
    return dic

    
def check_pos_mh(liste,types) :
    first_liste=[]
    if types=="start" :
        size=len(liste)
        i=0
        for elt in liste :
            if i==0 :
                first_match=int(elt[0])+int(elt[2])
                first_liste=elt
                i+=1
                continue
            else :
                other_match=int(elt[0])+int(elt[2])
                if other_match> first_match :
                    first_match=other_match
                    first_liste=elt
    if types=="end" :
        i=0
        for elt in liste :
            if i==0 :
                first_match=int(elt[0])
                first_liste=elt
                i+=1
                continue
            else :
                other_match=int(elt[0])
                if other_match< first_match :
                    first_match=other_match
                    first_liste=elt
    return first_liste
def clean_dic(dic):
    dic_cleanse=defaultdict(list)
    for elt in dic :
        liste_start=[]
        liste_end=[]
        for a in dic[elt] : 
            if "start" in a :
                liste_start.append(a)
            if "end" in a :
                liste_end.append(a)
        if len(liste_start)==1:
            liste_def_start=liste_start[0]
            dic_cleanse[elt].append(liste_def_start)
        if len(liste_end)==1 :
            liste_def_end=liste_end[0]
            dic_cleanse[elt].append(liste_def_end)
        if len(liste_start)>1 :
            liste_def_start=check_pos_mh(liste_start,"start")
            dic_cleanse[elt].append(liste_def_start)
        if len(liste_end)>1 :
            liste_def_end=check_pos_mh(liste_end,"end")
            dic_cleanse[elt].append(liste_def_end)
        #break
    return dic_cleanse
def concatenate_mh(dic_clean) :
    dic_size={}
    for elt in dic_clean :
        head=elt.split("_")[0]+"_"+elt.split("_")[1]+"_"+elt.split("_")[-1]
        if len(dic_clean[elt])==2 :
            if dic_clean[elt][-1]=="start":
                liste_start=dic_clean[elt][0]
                liste_end=dic_clean[elt][1]
            else :
                liste_end=dic_clean[elt][0]
                liste_start=dic_clean[elt][1]
            if int(liste_start[0])+int(liste_start[2])>=int(elt.split("_")[-1])-10 and int(liste_end[0])<=10 :
                if int(liste_start[0]) >= int(liste_end[0]) and int(liste_end[0])+int(liste_end[2])<=int(liste_start[0]) :
                    dic_size[head]=int(liste_start[2])+int(liste_end[2])
            else :
                if int(liste_start[2])> int(liste_end[2]) :
                    dic_size[head]=int(liste_start[2])
                else :
                    dic_size[head]=int(liste_end[2])
        elif len(dic_clean[elt])==1 :
            dic_size[head]=int(dic_clean[elt][0][2])
    return dic_size

In [ ]:
def fill_type(dic_small,dic_large,dic_all,dic_out) :
    dic_convert=get_convert(dic_all)
    for elt in dic_large :
        #dic_out[elt].append(dic_large[elt][0])
        dic_out[elt].append(dic_large[elt])

    for a in dic_small :
        #print(a)
        if a not in dic_all :
            head=get_converted(dic_convert,a)
        else :
            head=a
        if head not in dic_out :
            dic_out[head].append(dic_small[a])
    for v in dic_all :
        if v not in dic_out :
            dic_out[v].append(0)
    return dic_out
def Assign_type(dic,liste_type,types) :
    tmp=[]
    for elt in liste_type :
        if elt in dic :
            dic[elt].append(types)
            tmp.append(dic[elt][0])
    #print(tmp)
    return dic
def get_convert(dic) :
    dic_b=defaultdict()
    for elt in dic :
        head_dic=elt.split("_")[0]+"_"+elt.split("_")[1]
        dic_b[head_dic]=elt
    return dic_b
def get_converted(dic,a):
    head=a.split("_")[0]+"_"+a.split("_")[1]
    if head in dic :
        return dic[head]

In [ ]:

def concatenate_dic (dic_1,dic_4):
    concat_dic=defaultdict(list)
    for elt in dic_4 :
        #print(elt)
        head=elt.split("_")[0]+"_"+elt.split("_")[1]
        size=elt.split("_")[2]
        #print(elt,dic_1[head])
        total=[elt.split("_")[0],elt.split("_")[1],size]+dic_1[head]+dic_4[elt]
        #total.append(techno)
        #print(total)
        
        concat_dic[elt]=total
    return concat_dic
def Assign_techno(dic_all,dic_sr,techno_sr,techno_lr) :
    i=0
    for elt in dic_all :
        if elt in dic_sr :
            dic_all[elt].append(techno_sr)
            i+=1
        else :
            dic_all[elt].append(techno_lr)
    return dic_all